<a href="https://colab.research.google.com/github/Hase-U/learn_pytorch_advanced/blob/master/mycode/2_objectdetection/SSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 物体検出
commit用

```mycode/2_objectdetection/```



In [0]:
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import time
from google.colab import files

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## 2-1物体検出とは
**バウンディングボックス(bounding box)：**物体の位置を表す枠

インプット：画像

アウトプット：
- 画像のどこに物体が存在するかを示すバウンディングボックスの位置と大きさ情報
- 各バウンディングボックスに何が写っているか
- その検出に対する確信度

### SSDの流れ
デフォルトボックスの数がN, クラスが背景を入れてO+1の場合
1. 画像のリサイズ
2. デフォルトボックスを準備(N個)
3. 画像をSSDネットワークに入力(出力 N$\times$(O+1+4(オフセット)))
4. 信頼度上位のデフォルトボックスを抽出
5. オフセット情報による修正と被りの除去
6. 閾値を超える信頼度のもののみを最終出力にする

## 2-2 Datasetの実装
データセットがめちゃ大きくてダウンロードに15分毎回かかるから他にいい方法がないか以下を試した
- ```!wget```して解凍
- ファイルをアップロード
- Google drive でマウント

多分```!wget```を毎回するのが一番早いと思われ

In [0]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar xf VOCtrainval_11-May-2012.tar

In [0]:
!ls

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### 画像データ、アノテーションデータへのファイルパスリストを作成
**アノテーション**：正解データ。今回で言えば画像のどこに何があるかというバウンディングボックスの情報を指す

In [0]:
# 画像データとアノテーションデータへのファイルパスリストを作成する

def make_datapath_list(rootpath):

  # 画像ファイルとアノテーションファイルへのパスのテンプレートを作成
  imgpath_template = osp.join(rootpath, "JPEGImages", "%s.jpg")
  annopath_template = osp.join(rootpath, "Annotations", "%s.xml")

  #訓練と検証、それぞれのファイルのIDを取得する
  train_id_names = osp.join(rootpath+"ImageSets/Main/train.txt")
  val_id_names = osp.join(rootpath+"ImageSets/Main/val.txt")

  # 訓練データの画像ファイルとアノテーションファイルへのパスリストを作成
  train_img_list = list()
  train_anno_list = list()

  for line in open(train_id_names):
    file_id = line.strip()
    img_path = (imgpath_template % file_id) #画像パス
    anno_path = (annopath_template % file_id) #アノテーションのパス
    train_img_list.append(img_path)
    train_anno_list.append(anno_path)

    #検証データの画像ファイルとアノテーションファイルへのパスリストの作成
    val_img_list = list()
    val_anno_list = list()

  for line in open(val_id_names):
    # print(file_id)
    file_id = line.strip()
    img_path = (imgpath_template % file_id) #画像パス
    anno_path = (annopath_template % file_id) #アノテーションのパス
    val_img_list.append(img_path)
    val_anno_list.append(anno_path)   
  
  """
  それぞれで何が起きてるか確認用
  """
  # print("line : ", line)
  # print("file_id : ",file_id)
  # print("annopath_template : ",annopath_template)
  # print("anno_path : ",anno_path)

  return train_img_list, train_anno_list, val_img_list, val_anno_list

In [38]:
# 動作確認
rootpath = "./VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

print("----------------------------")

print(train_img_list[0])

line :  2011_003275

file_id :  2011_003275
annopath_template :  ./VOCdevkit/VOC2012/Annotations/%s.xml
anno_path :  ./VOCdevkit/VOC2012/Annotations/2011_003275.xml
----------------------------
./VOCdevkit/VOC2012/JPEGImages/2008_000008.jpg


In [0]:
!ls